In [2]:
%matplotlib inline

In [17]:
from keras.layers import merge, Convolution2D, MaxPooling2D, Input, Dense, Flatten, BatchNormalization, LeakyReLU
from keras.models import Model

nBottleneck = 2000

# first, define the encoder
inputs = Input(shape=(256, 256, 3))

# 256 x 256 input
x = Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same', dim_ordering='tf')(inputs)
x = BatchNormalization(mode=2)(x)
x = LeakyReLU(alpha=0.2)(x)

# 128 x 128 input
x = Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same', dim_ordering='tf')(x)
x = BatchNormalization(mode=2)(x)
x = LeakyReLU(alpha=0.2)(x)

# 64 x 64 input
x = Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same', dim_ordering='tf')(x)
x = BatchNormalization(mode=2)(x)
x = LeakyReLU(alpha=0.2)(x)

# 32 x 32 input
x = Convolution2D(128, 4, 4, subsample=(2, 2), border_mode='same', dim_ordering='tf')(x)
x = BatchNormalization(mode=2)(x)
x = LeakyReLU(alpha=0.2)(x)

# 16 x 16 input
x = Convolution2D(256, 4, 4, subsample=(2, 2), border_mode='same', dim_ordering='tf')(x)
x = BatchNormalization(mode=2)(x)
x = LeakyReLU(alpha=0.2)(x)

# 8 x 8 input
x = Convolution2D(512, 4, 4, subsample=(2, 2), border_mode='same', dim_ordering='tf')(x)
x = BatchNormalization(mode=2)(x)
x = LeakyReLU(alpha=0.2)(x)

# 4 x 4 input
encodedFeatures = Convolution2D(nBottleneck, 4, 4, subsample=(2, 2), border_mode='valid', dim_ordering='tf')(x)

encoder_model = Model(inputs, encodedFeatures)
encoder_model.summary()

# # then define the tell-digits-apart model
# digit_a = Input(shape=(1, 27, 27))
# digit_b = Input(shape=(1, 27, 27))

# # the vision model will be shared, weights and all
# out_a = vision_model(digit_a)
# out_b = vision_model(digit_b)

# concatenated = merge([out_a, out_b], mode='concat')
# out = Dense(1, activation='sigmoid')(concatenated)

# classification_model = Model([digit_a, digit_b], out)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_60 (Convolution2D) (None, 128, 128, 64)  3136        input_12[0][0]                   
____________________________________________________________________________________________________
batchnormalization_51 (BatchNorma(None, 128, 128, 64)  128         convolution2d_60[0][0]           
____________________________________________________________________________________________________
leakyrelu_51 (LeakyReLU)         (None, 128, 128, 64)  0           batchnormalization_51[0][0]      
___________________________________________________________________________________________